# Main Code (Run From Here)

## Initial Steps (used to get the start points, checkpoints and destination points)

In [1]:
# importing libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from math import atan2, cos, sin, sqrt, pi

def get_start_points(w, h):
    a = h / 10
    b1 = np.array([8.5*a, 0.5*a])
    b2 = b1 + np.array([a, 0])
    b3 = b2 + np.array([a, 0])
    b4 = b3 + np.array([a, 0])
    return [
        [int(x) for x in b1],
        [int(x) for x in b2],
        [int(x) for x in b3],
        [int(x) for x in b4]
    ]


def get_destinations(w, h):
    a = h / 10
    d1 = np.array([3*a//2, 17*a//2], np.uint8)
    d2 = d1 + np.array([0, a])
    d3 = d2 + np.array([17*a, 0])
    d4 = d3 + np.array([0, -a])
    return [
        [int(x) for x in d1],
        [int(x) for x in d2],
        [int(x) for x in d3],
        [int(x) for x in d4]
    ]


def get_checkpoints(w, h):
    a = h / 10
    c1 = np.array([17*a//2, 17*a//2], np.uint8)
    c2 = c1 + np.array([a, a])
    c3 = c2 + np.array([a, 0])
    c4 = c3 + np.array([a, -a])
    return [
        [int(x) for x in c1],
        [int(x) for x in c2],
        [int(x) for x in c3],
        [int(x) for x in c4]
    ]

# HSV Calculator (used to detect the two markers on the bot)

In [2]:
#HSV FINDER TOOL

def nothing(x):
    pass

def create_Trackbars(trackbarwindow = 'Controls',num_masks=1):
    
    lower = [0,0,0]
    upper = [180,255,255]
    #HUE
    cv2.createTrackbar('lh_mask1',trackbarwindow,lower[0],179,nothing)
    cv2.createTrackbar('uh_mask1',trackbarwindow,upper[0],179,nothing)
    
    #Saturation
    cv2.createTrackbar('ls_mask1',trackbarwindow,lower[1],255,nothing)
    cv2.createTrackbar('us_mask1',trackbarwindow,upper[1],255,nothing)
    
    #Value
    cv2.createTrackbar('lv_mask1',trackbarwindow,lower[2],255,nothing)
    cv2.createTrackbar('uv_mask1',trackbarwindow,upper[2],255,nothing)
    
    #Same for Mask2
    if num_masks==2:
        cv2.createTrackbar('lh_mask2',trackbarwindow,lower[0],179,nothing)
        cv2.createTrackbar('uh_mask2',trackbarwindow,lower[0],179,nothing)    

        cv2.createTrackbar('ls_mask2',trackbarwindow,lower[1],255,nothing)
        cv2.createTrackbar('us_mask2',trackbarwindow,lower[1],255,nothing)

        cv2.createTrackbar('lv_mask2',trackbarwindow,lower[2],255,nothing)
        cv2.createTrackbar('uv_mask2',trackbarwindow,lower[2],255,nothing)
    
    cv2.createTrackbar('save',trackbarwindow,0,1,nothing)
    #cv2.createTrackbar('mode',trackbarwindow,0,3,nothing)

def get_mask_3d(mask_number,hsv_img,trackbarwindow):#Here, 3d indicates that the shape of the mask is a tuple
    
    lh = cv2.getTrackbarPos('lh_mask{}'.format(mask_number),trackbarwindow)
    uh = cv2.getTrackbarPos('uh_mask{}'.format(mask_number),trackbarwindow)
    ls = cv2.getTrackbarPos('ls_mask{}'.format(mask_number),trackbarwindow)
    us = cv2.getTrackbarPos('us_mask{}'.format(mask_number),trackbarwindow)
    lv = cv2.getTrackbarPos('lv_mask{}'.format(mask_number),trackbarwindow)
    uv = cv2.getTrackbarPos('uv_mask{}'.format(mask_number),trackbarwindow)
    
    lower = np.array([lh,ls,lv])
    upper = np.array([uh,us,uv])
    mask = cv2.inRange(hsv_img,lower,upper)
    mask_3d = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)    
    
    return mask_3d,[lower,upper]

In [4]:
def get_hsv_limits(img,window_name="Controls"):
    cv2.namedWindow(window_name)
    create_Trackbars(window_name,num_masks=1)
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)   
    while True:
        mask,hsv_limits = get_mask_3d(1,hsv,window_name)
        mask_gray = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
        contours,h = cv2.findContours(mask_gray.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            max_area_cnt = max(contours, key = cv2.contourArea)
            peri = cv2.arcLength(max_area_cnt, True)
            approx = cv2.approxPolyDP(max_area_cnt, 0.02 * peri, True)
            pnts = approx.squeeze()

            img_contours = img.copy()
            cv2.drawContours(img_contours,[pnts],-1,(0,0,255),15)

        else:
            img_contours = img.copy()

        img_and_mask = np.hstack((img,mask)) 
        img_and_mask = cv2.resize(img_and_mask,None,fx = 0.4,fy=0.4)
        cv2.imshow(window_name,img_and_mask)
        key = cv2.waitKey(10)
        if key == ord('s'):
            break
    cv2.destroyAllWindows()
    
    return hsv_limits # [lower,upper]

# Other Helper Functions

### Finding centroids of the markers and finding their orientations

In [5]:
#returns the centroid of the detected marker on the bot which is later used to find the orientation
def get_centroid(bot_img,hsv_limits,draw_bb,marker_color):
    hsv_bot_img = cv2.cvtColor(bot_img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_bot_img,hsv_limits[0],hsv_limits[1])
    contours,h = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    marker_detected = False
    if contours:
        max_area_cnt = max(contours, key = cv2.contourArea)
        M = cv2.moments(max_area_cnt)
        if M['m00']==0:
            marker_detected = False
        else: 
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            if draw_bb:
                bot_img = cv2.drawContours(bot_img,[max_area_cnt],-1,marker_color,2)
            marker_detected = True
    
    if marker_detected == True:
        return (cx,cy)
    
    else: 
        return -1

In [6]:
def get_angle(img,previous_angle,hsv_limits1,hsv_limits,draw_bb = True):
    c1 = get_centroid(img,hsv_limits1,draw_bb,marker_color=(255,0,0))
    c2 = get_centroid(img,hsv_limits2,draw_bb,marker_color=(0,255,255))
    if c1 == -1 or c2 == -1:
        angle = previous_angle

    else:
        angle = round((atan2(c1[1]-c2[1],c2[0]-c1[0])*180/pi))

    return angle

### Get Command Function to give commands to the bot

In [7]:
from math import atan2,pi
def getcommand(loc, bot_orientation, dst):
    x,y = loc
    x1,y1 = dst
    target_orientation = (atan2((y-y1),(x1-x)))*180/pi
    diff_angle =  target_orientation-bot_orientation
    
    #converting negative difference in angle to positive
    if diff_angle<0:
        diff_angle += 360 

    #if the bot's orientation is within 20 degrees of the required orientation, we go straight
    if diff_angle <= 20:
        return 'F'
    
    #the difference angle is measured from the bots orientation vector to the destination direction vector
    #if it is within 180 degrees, the command is to take a left
    elif 20<diff_angle < 180:
        return 'L'
    else:
        return 'R'

### Used to warp an image when 4 points are given (used when cropping the arena)


In [8]:
def order_points(pts):
    #parameters: pts 4x2 np.ndarray
    #returns: 4x2 np.ndarray
    
    # The function initialises a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
#*************************************************************#


    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
#returns a warped image
#parameters: image (np.ndarray) and the 4 pts np.ndarray of size (4,2)

    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

# Execution

In [10]:
video1 = "Moving Bot.mp4"
video2 = "total_arcade2.mp4"
video3 = "total_arcade3.mp4"
video4 = "total_arcade_endless.mp4"

cap = cv2.VideoCapture(video1)
   
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video  file")

cv2.namedWindow('Frame')

grid_coordinates = []

def save_points(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        grid_coordinates.append((x,y))

################################# Locking and Cropping the Arena ###########################

print("Arena Locking Stage")
while(cap.isOpened()):
      
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
   
    # Display the resulting frame
        if len(grid_coordinates)!=0:
            for point in grid_coordinates:
                cv2.circle(frame, point, radius=2, color=(0,0,255), thickness=4)
        cv2.setMouseCallback('Frame', save_points)
        cv2.imshow("Frame",frame)

    
        key = cv2.waitKey(25)
        #clear the points if c is pressed
        if key == ord('c'):
            grid_coordinates =[]
        if key == ord('f'):
            w = frame.shape[1]
            h = frame.shape[0]
            grid_coordinates=[(0,0),(0,h),(w,0),(w,h)]
            break
        if key & 0xFF == ord('s'):
            break
        if key &0xFF == ord('p'): ##use this only for saving a photo after choosing 4 points 
            cv2.imwrite("grid_cropped{}.jpeg".format(np.random.randint(0,100)),four_point_transform(frame,np.array(grid_coordinates)))
            
   
  # Break the loop
    else: 
        break
cv2.destroyAllWindows()
print("Arena Locking Done\n")

################################### Selecting BOTS to be tracked #############################
if len(grid_coordinates)!=4:
    print("Didnt choose 4 points correctly. Choosing entire frame")
    cap.release()
    cv2.destroyAllWindows()

else:
    print("Drawing bounding boxes")
    TrDict = {'csrt': cv2.legacy.TrackerCSRT_create,
             'kcf' : cv2.legacy.TrackerKCF_create,
              }
    trackers = cv2.legacy.MultiTracker_create()

    ret,frame_roi = cap.read()
    frame = four_point_transform(frame_roi,np.array(grid_coordinates))
    bot_largest_dim = 0
    fraction = 0.9
    num_bots=1
    for i in range(num_bots):

        if ret:
            cv2.imshow('Frame',frame)
            bbi = cv2.selectROI('Frame',frame)
            bot_largest_dim = max(bbi[2:])
            tracker_i = TrDict['csrt']()
            trackers.add(tracker_i,frame,bbi)
            cv2.waitKey(1)
        else:
            print("Cap not reading in ROI Selection Stage")
            break
    bot_largest_dim = int(fraction*bot_largest_dim) #The extra region around the bot for detection of the bot
    
################################ HSV LIMITS SELECTION ##########################
    
    bot_img_t0 = frame[bbi[1]:bbi[1]+bbi[3],bbi[0]:bbi[0]+bbi[2]]
    bot_img_t0 = frame[:,:]
    cv2.imshow("bot_img_t0",bot_img_t0)
    hsv_limits1 = get_hsv_limits(bot_img_t0,"Finding HSV Limits of Marker 1")
    hsv_limits2 = get_hsv_limits(bot_img_t0,"Finding HSV Limits of Marker 2")
    
############################# Tracking and Velocity Publishing ##############################
    print("\nBOT Tracking...")
    angles = [-90 for i in range(num_bots)]
    bot_imgs = [0 for i in range(num_bots)]
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Not able to read in Tracking Stage")
            break
        frame = four_point_transform(frame,np.array(grid_coordinates))
        #frame_c = frame.copy()
        #cv2.imshow("orig",frame_c)
        (success, boxes) = trackers.update(frame)

        height,width,_ = frame.shape
        
        for i in range(len(boxes)):
            box = boxes[i]
            (x, y, w, h) = [int(a) for a in box]
            bot_centroid = (int(x + w/2), int(y + h/2))
            
            search_rect = [bot_centroid[0],bot_centroid[1],
                               bot_centroid[0]+2*bot_largest_dim,bot_centroid[1]+2*bot_largest_dim]
            
            frame_padded = np.stack([np.pad(frame[:,:,c], bot_largest_dim,
                        mode='constant', constant_values=0) for c in range(3)], axis=2)
            bot_imgs[i] = frame_padded[search_rect[1]:search_rect[3],search_rect[0]:search_rect[2]]

            angles[i] = get_angle(bot_imgs[i],angles[i],hsv_limits1,hsv_limits2,draw_bb = True)
            
            destination_point = (int(width/2),35)
            cv2.putText(frame,str(destination_point),destination_point,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            cv2.circle(frame,destination_point , radius=4, color=(0,0,255), thickness=4)
            
            #cv2.putText(frame,str(angles[i]),bot_centroid,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.circle(frame,bot_centroid , radius=4, color=(0,0,150), thickness=4)
            command = getcommand(bot_centroid,angles[i],destination_point)
            cv2.putText(frame,str(command),(bot_centroid[0]+10,bot_centroid[1]+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,150,0),2)
            

            print(command)
            ############CODE to Send to the server#############
            '''
            
            
            
            '''
            
        bot_img = np.hstack(bot_imgs) 
        cv2.imshow('Frame',frame)
        cv2.imshow('bot_img',bot_img)
        #cv2.imshow('bot_with_triangle',img_with_triangle)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Arena Locking Stage
Arena Locking Done

Drawing bounding boxes

BOT Tracking...
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
R
R
R
R
R
R
R
R
R
R
R
L
R
R
R
R
R
R
L
R
L
R
R
R
R
R
L
L
L
R
R
R
R
R
R
R
R
R
R
R
R
R
L
L
L
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
L
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
L
L
L
L
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R


# Other redundant/supplementary code (below)

# Video Reading Code And Saving Snapshots

In [21]:
import cv2 
import numpy as np
import time
# Create a video capture object, in this case we are reading the video from a file
vid_capture = cv2.VideoCapture("total_arcade_endless.mp4")

if (vid_capture.isOpened() == False):
    print("Error opening the video file")
else:

    fps = vid_capture.get(5)
    print('Frames per second : ', fps,'FPS')

# Get frame count
    # You can replace 7 with CAP_PROP_FRAME_COUNT as well, they are enumerations
    frame_count = vid_capture.get(7)
    print('Frame count : ', frame_count)
    vid_capture.set(cv2.CAP_PROP_FPS, 1)
    fps = vid_capture.get(5)
    print('Frames per second : ', fps,'FPS')
while(vid_capture.isOpened()):
    # vid_capture.read() methods returns a tuple, first element is a bool 
    # and the second is frame
    ret, frame = vid_capture.read()
    if ret == True:
        cv2.imshow('Frame',frame)
        # 20 is in milliseconds, try to increase the value, say 50 and observe
        key = cv2.waitKey(20)
        
       # time.sleep(0.1)
        if key == ord('q'):
            break
        if key == ord('s'):
            cv2.imwrite("triangle{}.jpeg".format(np.random.randint(0,100)),frame)
            break
    else:
        break

# Release the video capture object
vid_capture.release()
cv2.destroyAllWindows()

Frames per second :  30.0 FPS
Frame count :  988.0
Frames per second :  30.0 FPS


# Arena Locking Tool and Mapping (Ignore this)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#HSV FINDER TOOL

def nothing(x):
    pass

def create_Trackbars(trackbarwindow = 'Controls'):
    
    lower = [0,0,0]
    upper = [180,255,255]
    #HUE
    cv2.createTrackbar('lh_mask1',trackbarwindow,lower[0],179,nothing)
    cv2.createTrackbar('uh_mask1',trackbarwindow,upper[0],179,nothing)
    
    #Saturation
    cv2.createTrackbar('ls_mask1',trackbarwindow,lower[1],255,nothing)
    cv2.createTrackbar('us_mask1',trackbarwindow,upper[1],255,nothing)
    
    #Value
    cv2.createTrackbar('lv_mask1',trackbarwindow,lower[2],255,nothing)
    cv2.createTrackbar('uv_mask1',trackbarwindow,upper[2],255,nothing)
    
    #Same for Mask2
    cv2.createTrackbar('lh_mask2',trackbarwindow,lower[0],179,nothing)
    cv2.createTrackbar('uh_mask2',trackbarwindow,lower[0],179,nothing)    
    
    cv2.createTrackbar('ls_mask2',trackbarwindow,lower[1],255,nothing)
    cv2.createTrackbar('us_mask2',trackbarwindow,lower[1],255,nothing)
    
    cv2.createTrackbar('lv_mask2',trackbarwindow,lower[2],255,nothing)
    cv2.createTrackbar('uv_mask2',trackbarwindow,lower[2],255,nothing)
    
    cv2.createTrackbar('save',trackbarwindow,0,1,nothing)
    #cv2.createTrackbar('mode',trackbarwindow,0,3,nothing)

def get_mask_3d(mask_number,hsv_img,trackbarwindow):#Here, 3d indicates that the shape of the mask is a tuple
    
    lh = cv2.getTrackbarPos('lh_mask{}'.format(mask_number),trackbarwindow)
    uh = cv2.getTrackbarPos('uh_mask{}'.format(mask_number),trackbarwindow)
    ls = cv2.getTrackbarPos('ls_mask{}'.format(mask_number),trackbarwindow)
    us = cv2.getTrackbarPos('us_mask{}'.format(mask_number),trackbarwindow)
    lv = cv2.getTrackbarPos('lv_mask{}'.format(mask_number),trackbarwindow)
    uv = cv2.getTrackbarPos('uv_mask{}'.format(mask_number),trackbarwindow)
    
    lower = np.array([lh,ls,lv])
    upper = np.array([uh,us,uv])
    mask = cv2.inRange(hsv_img,lower,upper)
    mask_3d = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)    
    
    return mask_3d

In [ ]:
def visualise_contours(img,contours,thickness=3):
    img_c = img.copy()
    cv2.drawContours(img_c,contours,-1,(255,0,0),thickness)
    plt.imshow(img_c,cmap='gray')
    plt.show()
    return img_c

def get_points_for_warping(qmask):
    
    qmask_gray = cv2.cvtColor(qmask,cv2.COLOR_BGR2GRAY)
    contours,h = cv2.findContours(qmask_gray.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    max_area_cnt = max(contours, key = cv2.contourArea)
    peri = cv2.arcLength(max_area_cnt, True)
    approx = cv2.approxPolyDP(max_area_cnt, 0.02 * peri, True)
    pnts = approx.squeeze()
    
    if pnts.shape !=(4,2):
        visualise_contours(img,[contours])
    return pnts,contours

In [ ]:
def order_points(pts):
    #parameters: pts 4x2 np.ndarray
    #returns: 4x2 np.ndarray
    
    # The function initialises a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
#*************************************************************#


    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
#returns a warped image
#parameters: image (np.ndarray) and the 4 pts np.ndarray of size (4,2)

    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [7]:
def initiate_mapping(cap):
    
    if cap.isOpened()==False:
        print('Could not access feed')
        return
    
    trackbarwindow = 'Controls'
    cv2.namedWindow(trackbarwindow)
    create_Trackbars(trackbarwindow)
    
    pnts = np.array([[0,0],[0,1],[1,0],[1,1]]) 
    while True:

        ret,img = cap.read()
        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        save = cv2.getTrackbarPos('save',trackbarwindow)


        mask1 = get_mask_3d(1,hsv_img,trackbarwindow)
        mask2 = get_mask_3d(2,hsv_img,trackbarwindow)
        mask = cv2.bitwise_or(mask1,mask2)

        mask1_and_mask2 = np.hstack((mask1,mask2))
        img_and_mask = np.hstack((img,mask))
        
        stacked = np.vstack((img_and_mask,mask1_and_mask2))
        stacked = cv2.resize(stacked,None,fx=0.25,fy=0.25)
        cv2.imshow("Mask",stacked)
        
        mask_gray = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
        contours,h = cv2.findContours(mask_gray.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            max_area_cnt = max(contours, key = cv2.contourArea)
            peri = cv2.arcLength(max_area_cnt, True)
            approx = cv2.approxPolyDP(max_area_cnt, 0.02 * peri, True)
            pnts = approx.squeeze()

            img_contours = img.copy()
            cv2.drawContours(img_contours,[pnts],-1,(0,0,255),15)
            img_contours = cv2.resize(img_contours,None,fx = 0.7,fy=0.7)
            cv2.imshow('Grid Recognition',img_contours)
        
        else:
            cv2.imshow('Grid Recognition',img.copy())


        warped = four_point_transform(img,pnts)
        warped = cv2.resize(warped,None,fx = 0.7,fy=0.7)
        cv2.imshow('Warped',warped)
        key = cv2.waitKey(10)
        
        if save ==1:
            mask_no = np.random.randint(1,10000)
            cv2.imwrite('mask_{}.jpg'.format(mask_no),mask)
            break
        if key == ord('s'):
            break
            
    cv2.destroyAllWindows()
    while cap.isOpened():
        ret,img = cap.read()
        
        warped = four_point_transform(img,pnts)
        cv2.imshow('Grid',warped)
        gray_warped = cv2.cvtColor(warped,cv2.COLOR_BGR2GRAY)
        #ret,binary_warped = cv2.threshold(gray_warped,200,255,cv2.THRESH_BINARY)
        gray_warped = cv2.resize(gray_warped, None,fx=2,fy=2)
        cv2.imshow('Grid_Binary',gray_warped)
        if cv2.waitKey(10)==ord('q'):
            break
        
    cv2.destroyAllWindows()
    cap.release()

In [ ]:
cap = cv2.VideoCapture(0)
cap.isOpened()

In [ ]:
initiate_mapping(cap)

In [16]:
cv2.destroyAllWindows()
cap.release()

In [ ]:
def get_orientation_rough2(bot_img):
    gray = cv2.cvtColor(bot_img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)

    (T, mask) = cv2.threshold(blurred, 200, 255,cv2.THRESH_BINARY_INV)
    contours,h = cv2.findContours(255-mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    img_contours = bot_img.copy()
    if contours:
        max_area_cnt = max(contours, key = cv2.contourArea)
        peri = cv2.arcLength(max_area_cnt, True)
        approx = cv2.approxPolyDP(max_area_cnt, 0.02 * peri, True)
        pnts = approx.squeeze()
        cv2.drawContours(img_contours,[pnts],-1,(0,0,255),3)

   # angle = round(getOrientation(max_area_cnt, img_contours)*180/3.141)
    return img_contours#,angle

In [ ]:
def get_orientation_rough1(bot_img):
    gray = cv2.cvtColor(bot_img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)

    (T, mask) = cv2.threshold(blurred, 200, 255,cv2.THRESH_BINARY_INV)
    contours,h = cv2.findContours(255-mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    max_area_cnt = max(contours, key = cv2.contourArea)

    img_c = bot_img.copy()
    cv2.drawContours(img_c,max_area_cnt,-1,(255,0,0),2)
    angle = round(getOrientation(max_area_cnt, img_c)*180/3.141)
    return angle,img_c

In [51]:
(int(w/2),int(h/2))

(8, 22)

In [52]:
h,w

(44, 17)

In [38]:
cv2.destroyAllWindows()
cap.release()

In [ ]:
hsv_limits1 = get_hsv_limits(img,"Finding HSV Limits of Marker 1")
hsv_limits2 = get_hsv_limits(img,"Finding HSV Limits of Marker 2")

In [ ]:
plt.imshow(img)
get_angle(img,0,hsv_limits1,hsv_limits2)

In [ ]:
ret,thresh = cv2.threshold(img,127,255,0)
print(thresh.shape)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
mask = cv2.inRange(hsv_img,)
mask_3d = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR) 

In [12]:
import cv2
import numpy as np

In [50]:
img = cv2.imread("grid_cropped5.jpeg")
cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [52]:
w = img.shape[1]
h = img.shape[0]

get_start_points(w,h)
get_checkpoints(w,h)
get_destinations(w,h)

[[21, 124], [21, 138], [269, 138], [269, 124]]

In [53]:
for i, b in enumerate(get_start_points(w, h)):
    x, y = b
    cv2.circle(img, (x, y), 2, (255, 0, 0), -1)
    cv2.putText(img, f's{i+1}', (x+1, y+1),
                cv2.FONT_HERSHEY_SIMPLEX, 0.25, (255, 0, 0), 1)

for i, c in enumerate(get_checkpoints(w, h)):
    x, y = c
    cv2.circle(img, (x, y), 2, (0, 0, 255), -1)
    cv2.putText(img, f'c{i+1}', (x+1, y+1),
                 cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 255), 1)

for i, d in enumerate(get_destinations(w, h)):
    x, y = d
    cv2.circle(img, (x, y), 2, (0, 255, 0), -1)
    cv2.putText(img, f'd{i+1}', (x+1, y+1),
               cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 255, 0), 1)

#cv2.imwrite('RESULT.png', img)
cv2.imshow('Platform with points', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
getcommand((0,10),180,(10,10))

'R'

# Pose Estimation using triangle marker

Getting Orientation from Bot Image

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from math import atan2, cos, sin, sqrt, pi

In [ ]:
def get_pose(img,previous_angle,bot_centroid):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    (T, mask) = cv2.threshold(blurred,230,255,cv2.THRESH_BINARY)

    contours,h = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    img_contours =  img.copy()
    detected = False
    if len(contours)!=0:
        required_pnts=[]
        max_area = 0
        for cnt in contours:
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt, 0.09 * peri, True)
            area = cv2.contourArea(approx)
            pnts = approx.squeeze()  #np.ndarray (n,1 size)
            if len(pnts)==3:
                if area<max_area:
                    pass
                else:
                    detected = True
                    required_pnts = pnts
                    max_area = area
        
    
    if detected:
        cv2.drawContours(img_contours,[required_pnts],-1,(0,0,255),1)
        max_norm = 0 
        max_points = []
        l1 = norm(required_pnts[0]-required_pnts[1])
        l2 = norm(required_pnts[1]-required_pnts[2])
        l3 = norm(required_pnts[2]-required_pnts[0])
        if l1<=l2 and l1<=l3:
            mid_point = (required_pnts[0]+required_pnts[1])/2
            top_point = required_pnts[2]

        if l2<=l1 and l2<=l3:
            mid_point = (required_pnts[1]+required_pnts[2])/2
            top_point = required_pnts[0]

        if l3<=l1 and l3<=l2:
            mid_point = (required_pnts[2]+required_pnts[0])/2
            top_point = required_pnts[1]

        mid_point = mid_point.astype(np.uint32) 
        angle = round(atan2(mid_point[1]-top_point[1],top_point[0]-mid_point[0])*180/pi)
        centroid = (mid_point+top_point)/2
        centroid = centroid.astype(np.uint32)
    
    else:
        angle= previous_angle
        centroid = bot_centroid
    return angle,centroid,img_contours

In [ ]:
img = cv2.imread("5x5.png")
img_text = img.copy()
plt.imshow(img)
angle, centroid,img_contours = get_pose(img,0,(10,10))
cv2.putText(img_text,str(angle),(10000,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
plt.imshow(img_contours)
plt.figure()
plt.imshow(img_text)

In [ ]:
cv2.waitKey(0)
cv2.destroyAllWindows()

# Code to Determine Orientation using PCA

In [ ]:
from math import atan2, cos, sin, sqrt, pi

def drawAxis(img, p_, q_, color, scale):
    p = list(p_)
    q = list(q_)
 
    ## [visualization1]
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
 
    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
 
    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
 
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
    ## [visualization1]

def getOrientation(pts, img):
    ## [pca]
    # Construct a buffer used by the pca analysis
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
 
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
 
    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
     
    ## [pca]
    ## [visualization]
    # Draw the principal components
    #cv2.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    drawAxis(img, cntr, p1, (255, 255, 0), 1)
    drawAxis(img, cntr, p2, (0, 0, 255), 5)
 
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    ## [visualization]
     # Label with the rotation angle
  
    #label = str(-int(np.rad2deg(angle)) - 90) + " degrees"
    #textbox = cv2.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
    #cv2.putText(img, label, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
 
    return angle

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
   
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video  file")

while(cap.isOpened()):
      
  # Capture frame-by-frame
    ret, img = cap.read()
    if ret == True:
   
    # Display the resulting frame
        cv2.imshow("Frame",img)
        
        # Convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Convert image to binary
        _, bw = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        # Find all the contours in the thresholded image
        contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)


        for i, c in enumerate(contours):

          # Calculate the area of each contour
            area = cv2.contourArea(c)

          # Ignore contours that are too small or too large
            if area < 3700 or 100000 < area:
                continue

          # Draw each contour only for visualisation purposes
            cv2.drawContours(img, contours, i, (0, 0, 255), 2)

          # Find the orientation of each shape
            #getOrientation(c, img)

        cv2.imshow('Output Image', img)
    
        key = cv2.waitKey(25)
        
        if key & 0xFF == ord('q'):
            break
   
  # Break the loop
    else: 
        break
cv2.destroyAllWindows()

In [ ]:
# Load the image
img = cv2.imread("bot.png")
 
# Was the image there?
if img is None:
    print("Error: File not found")
    exit(0)

cv2.imshow('Input Image', img)
 
# Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
# Convert image to binary
_, bw = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
 
# Find all the contours in the thresholded image
contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
 

for i, c in enumerate(contours):
 
  # Calculate the area of each contour
    area = cv2.contourArea(c)
 
  # Ignore contours that are too small or too large
    if area < 3700 or 100000 < area:
        continue
 
  # Draw each contour only for visualisation purposes
    cv2.drawContours(img, contours, i, (0, 0, 255), 2)
 
  # Find the orientation of each shape
    #getOrientation(c, img)

cv2.imshow('Output Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()